In [3]:
with open('data/songs/寻找独角兽.txt') as file:
    content = file.read().splitlines()
    
file.close()

clean_l = [x for x in content if x != '' and 'https' not in x]

In [4]:
# Split chinese and english
cn_l, en_l = clean_l[::2], clean_l[1::2]

In [7]:
# Split tones
import string

dict_table = str.maketrans('', '', string.digits)
clean_en_l = [x.translate(dict_table) for x in en_l]

In [8]:
from termcolor import colored

red = "\033[31m"
green = "\033[32m"
reset = "\033[39m"

word_book = {}
word_info = []

for s in range(2):
    print(cn_l[s])
    
    i = input()

    correct_cn_w = list(cn_l[s].replace(' ', ''))
    correct_words = clean_en_l[s].split(' ')
    colored_correct_words = correct_words.copy()
    check_words = i.split(' ')

    for w in range(len(check_words)):

        if (check_words[w] != correct_words[w]) and ('/' not in correct_words[w] or check_words[w] not in correct_words[w].split('/')):
            check_words[w] = red + check_words[w] + reset
            colored_correct_words[w] = green + correct_words[w] + reset

            if correct_cn_w[w] in word_book:
                word_book[correct_cn_w[w]][1] += 1
            else:
                word_book[correct_cn_w[w]] = [correct_words[w], 1]

    output_lyrics = ' '.join(check_words)
    correct_lyrics = ' '.join(colored_correct_words)    

    print(correct_lyrics)
    print(output_lyrics)

树上大鸟 与风神在说话
syu soeng daai niu jyu fung san zoi syut waa
sy\
地上巨兽 寄居解语花下
dei soeng geoi sau gei geoi gaai jyu/jyu faa haa



In [13]:
# output json word book to file
import json

with open('data/new words.json', 'w') as f:
    json.dump(word_book, f, indent=4, ensure_ascii=False)

f.close()

In [15]:
import pandas as pd

# load json word book from file
f = open('data/new words.json', 'r')

js_wb = json.loads(f.read())
js_wb

{'上': ['soeng', 2], '与': ['jyu', 1]}

In [61]:
# output as dataframe
data = [[key] + values for key, values in js_wb.items()]
df = pd.DataFrame(data, columns=['Word', 'Pronunciation', 'Counts'])
df

,Word,Pronunciation,Counts
0,上,soeng,2
1,与,jyu,1


In [20]:
# check existing
import os

if not os.path.isfile('data/new words.txt'):
    open('data/new words.txt', 'a').close()
else:
    print('New words book is existing.')

New words book is existing.


In [24]:
# get list of exisitng songs
import glob

all_f = glob.glob('data/songs/*.txt')
all_songs = [x[11:-4] for x in all_f]
all_songs

['再见二丁目',
 '南昌街王子',
 '爱与诚',
 '钟无艳',
 '南瓜车',
 '爱得太迟',
 '富士山下',
 '甜蜜蜜',
 '我们都是这样长大的',
 '恋爱大过天',
 '红绿灯',
 '寻找独角兽',
 '一丝不挂',
 '黄色大门',
 '你们的幸福',
 '下一站天后',
 '饮歌',
 '年度之歌',
 '心淡',
 '简爱',
 '喜帖街',
 '灰姑娘',
 '男孩像你',
 '千金',
 '谁愿放手',
 '勇',
 '死性不改']

In [31]:
title_l1 = "Let's start to learn Cantonese!!!"

game_start = f"+{'-'*(20+len(title_l1))}+\n|{' '*(20+len(title_l1))}|\n|{' '*10}{title_l1}{' '*10}|\n|{' '*(20+len(title_l1))}|\n+{'-'*(20+len(title_l1))}+\n"
print(game_start)

+-----------------------------------------------------+
|                                                     |
|          Let's start to learn Cantonese!!!          |
|                                                     |
+-----------------------------------------------------+



In [2]:
for n in range(23):
    def print_n(n):
        print(n+1) 



In [8]:
import tkinter as tk

root = tk.Tk()      

root.mainloop()

In [6]:
#!/usr/bin/env python3

import tkinter as tk
from tkinter.font import Font

class Pad(tk.Frame):

    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)

        self.toolbar = tk.Frame(self, bg="#eee")
        self.toolbar.pack(side="top", fill="x")

        self.bold_btn = tk.Button(self.toolbar, text="Bold", command=self.make_bold)
        self.bold_btn.pack(side="left")

        self.clear_btn = tk.Button(self.toolbar, text="Clear", command=self.clear)
        self.clear_btn.pack(side="left")

        # Creates a bold font
        self.bold_font = Font(family="Helvetica", size=14, weight="bold")

        self.text = tk.Text(self)
        self.text.insert("end", "Select part of text and then click 'Bold'...")
        self.text.focus()
        self.text.pack(fill="both", expand=True)

        # configuring a tag called BOLD
        self.text.tag_configure("BOLD", font=self.bold_font)

    def make_bold(self):
        # tk.TclError exception is raised if not text is selected
        try:
            self.text.tag_add("BOLD", "sel.first", "sel.last")        
        except tk.TclError:
            pass

    def clear(self):
        self.text.tag_remove("BOLD",  "1.0", 'end')


def demo():
    root = tk.Tk()
    Pad(root).pack(expand=1, fill="both")
    root.mainloop()


if __name__ == "__main__":
    demo()

In [5]:
from tkinter import *

root = Tk()

txt = Text(root)
txt.pack()

s = ['white', 'white', 'red', 'red', 'white', 'white', 'white', 'white']

txt.tag_config('warning', foreground="red")

for i in s:
    if i == 'white':
        txt.insert('insert', f'{i} ')
    else:
        txt.insert('insert', i, 'warning')
        txt.insert('insert', ' ')

root.mainloop()

In [4]:
from tkinter import *
from tkinter import font

root = Tk()

fontsize = 16

def increase_font():
    global fontsize
    fontsize += 2
    textfont.config(size=fontsize)

textfont = font.Font(family = "consolas" , size = fontsize)
# set the width and height of the Text widget to smallest values
# and let the layout manager to expand it
my_text = Text(root , width=1 , height=1 , font=textfont)
my_text.pack(fill="both", expand=1) # fill the parent frame
my_text.insert(1.0 , "There is a problem that use to test if the text widge is fixed. If the line goes to the next line, it will be correct.")

my_button = Button(root , text="Increase Font" , width=13 , font="arial 11" , command=increase_font)
my_button.pack(pady=5)

mainloop()

In [10]:
from tkinter import *
wind=Tk()
wind.geometry('450x450')
l1=Label(wind,text='Did you mean:')
l1.pack(side=TOP, anchor=NW)
b1=Button(wind,text='Button1',width=450)
b1.pack()
b2=Button(wind,text='Button2',width=450)
b2.pack()
wind.mainloop()